# 1. ARIMA
- 전통적인 통계 분석 기법 / ML, DL 분석법 아니다.
- Auto Regressive Integrated Moving Average
- AutoRegressive + MovingAverage
- 정상성 : 평균, 분산이 시간이 지남에 따라 일정해지는 성질
- 시계열 데이터 중 변동폭이 일정하지 않거나 추세, 계절성이 있는 경우 로그 변환, 차분으로 정상성 확보해준다
- 계절성 O or 계절성 X로 ARIMA 분석 다르게 진행
    - 계절성 : 아이스크림 판매량
    - 비계절성 : 주가
- refer : https://leedakyeong.tistory.com/entry/Python-%EB%82%A0%EC%94%A8-%EC%8B%9C%EA%B3%84%EC%97%B4-%EB%8D%B0%EC%9D%B4%ED%84%B0Kaggle%EB%A1%9C-ARIMA-%EC%A0%81%EC%9A%A9%ED%95%98%EA%B8%B0


In [1]:
# import
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose #Trend, Cycle, Seasonal, Random 네 요소로 데이터 추이 그래프 보여준다.
import statsmodels.api as sm #ACF 그래프 그리기
from statsmodels.tsa.stattools import adfuller


import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')


In [ ]:
### 1. 시계열 분해법으로 데이터가 계절성 같은 패턴을 보이는지 파악해야한다.
# 데이트타임을 인덱스로 설정한 데이터 만들어줍니다.
path = 'data/df.csv'
df = pd.read_csv(path, index_col=0)
time = df.loc[:, ['date', 'value']]
time.index = time.date
ts = time.drop('date', axis=1)
ts

In [ ]:
result = seasonal_decompose(ts['value'], model='additive', freq=7)
# model =
# freq = 4(분기), 12(월별), 7(주별), 356(일별)

In [ ]:
# 데이터가 패턴을 보이면 정상성 부합하는지 확인해봐야한다.
# ACF 그래프 그려보기 : 패턴 유무를 파악하는 그래프라 이해
# 그래프가 천천히 줄어들면 정상성을 만족하지 않는다는걸 의미한다.

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(ts, lags=20, ax=ax)

In [ ]:
# 데이터가 패턴을 보이면 정상성 부합하는지 확인해봐야한다.
# PACF 그래프 그려보기 : 직접적으로 영향 주는 부분만 계산해 패턴 유무 파악하는 그래프
# 그래프가 천천히 줄어들면 정상성을 만족하지 않는다는걸 의미한다.

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_pacf(ts, lags=20, ax=ax)

In [ ]:
# 정상성 여부 숫자로 확인 :: ADF test
# p-value가 0.05 이하면 정상성 OK
# p-value가 0.05보다 크면 정상성 X
# ARIMA 분석 위해선 정상성 있어야

result = adfuller(ts)
print('ADF statistic : {}'.format(result[0]))
print('p-value : {}'.format(result[1]))
print('Critical Values :')
for key, value in result[4].items():
    print('\t{0}% : {1}'.format(key, value))

In [ ]:
# 차분 : 연이은 관측값들의 차이를 계산하는 것

ts_diff = ts - ts.shift()
plt.figure(figsize=(20,10))
plt.plot(ts_diff)
plt.title('Differencing Method')
plt.xlabel('Date')
plt.ylabel('Differencing Mean Temp')
plt.show()

In [ ]:
# 차분한 데이터로 정상성 검사해보기
result = adfuller(ts_diff[1:])
print('ADF statistic : {}'.format(result[0]))
print('p-value : {}'.format(result[1]))
print('Critical Values :')
for key, value in result[4].items():
    print('\t{0}% : {1}'.format(key, value))

In [ ]:
# 아리마 모델에 fit해보기
# 아리마모델은 (p,q,d) 값으로 여러개를 정함
# 정하는 방식은 ACF와 PACF가 0에 수렴하는 lag값 보며 결정

#fit model
model = ARIMA(ts, order=(2,1,2))
model_fit = model.fit(disp=0)

#predict
start_index = dt(2020, 3, 1)
end_index = dt(2022, 5, 2)
forecast = model_fit.predict(start=start_index, end=end_index, typ = 'levels')
# 차분이 들어간 모델에서 typ를 default값인 'linear'로 설정하면 차분한 값에 대한 결과가 나온다.
# 해당 모델에서 알고 싶은 결과는 차분하기 전의 데이터에 대한 시계열 분석이다. 따라서 typ='levels'로 지정해준다.

#visualization
plt.figure(figsize=(20,10))
plt.plot(df.date, df.value, label='original')
plt.plot(forecast, label='predicted')
plt.title("Time Series Forecast")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.show()

In [ ]:
# 마지막 잔차 분석을 통해 모델에 빠진 것이 없는지 확인
# 잔차는 어떠한 패턴이나 특성이 나타나선 안된다.

resi = np.array(df[df.date >= start_index].value) - np.array(forecast)
# 예측 범위 맞춰서 실제값과 예측값을 빼준 것

plt.figure(figsize=(20,10))
plt.plot(df.date[df.date >= start_index], resi) #x축은 날짜 / y축은 잔차값
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.show()

In [ ]:
# 잔차 ACF 그래프와 ADF 검정 통해 마지막 점검

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_pacf(resi, lags=20, ax=ax)

In [ ]:
result = adfuller(resi)
print('ADF statistic : {}'.format(result[0]))
print('p-value : {}'.format(result[1]))
print('Critical Values :')
for key, value in result[4].items():
    print('\t{0}% : {1}'.format(key, value))

In [ ]:
# 성능 확인

from sklearn import metrics

def scoring(y_true, y_pred):
    r2 = round(metrics.r2_score(y_true, y_pred) * 100, 3)
    corr = round(np.corrcoef(y_true, y_pred)[0, 1], 3)
    mape = round(metrics.mean_absolute_percentage_error(y_true, y_pred) * 100, 3)
    rmse = round(metrics.mean_squared_error(y_true, y_pred, squared=False), 3)

    df = pd.DataFrame({
        'R2':r2,
        'Corr':corr,
        'MAPE':mape,
        'RMSE':rmse
    })

score = scoring(df[df.date >= start_index].value, forecast)